# Langue des signes : réseaux de neurones

## Keras et Tensorflow

Pour installer Keras et Tensorflow sans GPU :

Pour installer la version GPU sous windows, cf https://medium.com/@raza.shahzad/setting-up-tensorflow-gpu-keras-in-conda-on-windows-10-75d4fd498198  
Sous Linux : http://deeplearning.lipingyang.org/2017/08/01/install-keras-with-tensorflow-backend/  
Sous MacOS (avec GPU Nvidia) : https://blog.wenhaolee.com/run-keras-on-mac-os-with-gpu/

## Initialisations

In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import model_selection

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

from sklearn import datasets

In [ ]:
from keras.datasets import mnist

from keras.models import Sequential, load_model

from keras.layers import Dense, Dropout, Flatten

from keras.layers.convolutional import Conv2D, MaxPooling2D

from keras.utils.np_utils import to_categorical

## Le dataset du langage des signes

On utilise le dataset du langage des signes :  
https://www.kaggle.com/datamunge/sign-language-mnist#american_sign_language.PNG

<img src="https://storage.googleapis.com/kagglesdsdata/datasets/3258/5337/amer_sign2.png?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1582898150&Signature=FSNN2mBuff7zGjEYUvGBn%2FrLFoYAGfkl5ivl6PT127E7NEAAnfxyWKOLtSpRN3ZN4NbBoXhzo%2Bee%2F5rC%2F7rnI1SyTljAIzjqcE9%2BMINHU3IFBJqJErsH5ilIln2d73QG%2BMXz8F3jGezxjISP%2BGa3SD7WC13og1hpRHMjS0bo4teSkZCRNycHUaCQV16ZRGmhpgftAuhBxPUFZBqTt46nMpnG5pYpl6NYRAD4ss3%2B7BvmPS%2FUMzFwDCeZsNLq0VNcqpl05isQZ2qT1KhFdg1j0K0zp%2BO%2BIALp42iselnVsx%2B1cc97YsELZd1t6mIp6TrTUQ06HndlxSw9J1FilaQKdg%3D%3D">

In [ ]:
df=pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')

In [ ]:
df.shape

In [ ]:
df.head()

Le dataset est bien équilibré (mais il manque les lettres Y et Z) :

In [ ]:
df.label.value_counts()

In [ ]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
print(labels)

On convertit les lignes de pixels en matrices (images) :

In [ ]:
n_samples = len(df.index)
images = np.array(df.drop(['label'],axis=1))
images = images.reshape(n_samples,28,28)

On affiche les 50 premiers :

In [ ]:
plt.figure(figsize=(10,20))
for i in range(0,49) :
    plt.subplot(10,5,i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap="gray_r")
    plt.title(labels[df.label[i]])

## Réseaux denses (sklearn)

On sépare la cible et les caractéristiques :

In [ ]:
y = df['label']
X = df.drop(['label'] , axis=1)

On normalise les valeurs entre 0 et 1 :

In [ ]:
X = X/255

Séparation train / test :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

On utilise la méthode *MLPClassifier* de *sklearn* pour utiliser un réseau de neurones à deux couches cachées de 200 et 60 neurones :

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(200,60))
mlp.fit(X_train,y_train)
y_mlp = mlp.predict(X_test)

La pertinence est très bonne :

In [ ]:
mlp_score = accuracy_score(y_test, y_mlp)
print(mlp_score)

et la matrice de confusion :

In [ ]:
pd.crosstab(y_test, y_mlp, rownames=['Reel'], colnames=['Prediction'], margins=True)

## Réseaux denses (Keras/Tensorflow)

La définition du même réseau de neurones est un peu plus compliquée avec *Keras*, mais il est utile de comprendre cette première étape avant d'utiliser les réseaux convolutifs

Comme l'activation d'un neurone donne une valeur (probabilité) entre 0 ou 1, on code la cible (classes entre 0 et 24) sous la forme d'un vecteur de 0 ou 1 (*one hot encoding*) avec *to_categorical* :

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
print(y[0])
y_cat = to_categorical(y)
print(y_cat[0])

On a seulement 25 classes (il manque les Y et Z dans le dataset) :

In [ ]:
num_classes = y_cat.shape[1]
print(num_classes)

On sépare train et test :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=1)

Pour Keras, il est nécessaire d'avoir des tableaux et non des dataframes :

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

On va utiliser une architecture en *couches* (modèle *Sequential*), avec des couches *denses* :

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

On définit un modèle à deux couches cachées de 200 et 60 neurones  
La dernière couche comporte 25 neurones (le nombre de classes) pour la classification :

In [ ]:
model = Sequential()
model.add(Dense(200, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

L'activation de la dernière couche est un *softmax* : la somme des valeurs de l'activation des neurones de la dernière couche est 1  
(on interprète la sortie des derniers neurones comme une probabilité d'appartenance à la classe correspondante)

On "compile" le modèle, avec une *categorical_crossentropy* comme mesure de distance (distance probabiliste multi classes)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train = model.fit(X_train , y_train , validation_data=(X_test,y_test), epochs=30, verbose=1)

Pertinence :

In [ ]:
model.evaluate(X_test,y_test)

La variable *train* mémorise l'historique des scores sur l'ensemble d'apprentissage :

In [ ]:
print(train.history['accuracy'])

et sur l'ensemble de validation :

In [ ]:
print(train.history['val_accuracy'])

On définit une fonction pour afficher un graphique des scores :

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

## Exercice : tester les réseaux de neurones sur le dataset *fashion_MNIST*

## Réseaux denses (sklearn)

Chargement des données

In [ ]:
df = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')

Préparation des données d'entrainement et de test.
Conversion des données DataFrame en tableaux.

In [ ]:
labels = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt",
          "Sneaker","Bag","Ankle boot"]
y = df['label']
X = df.drop(['label'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

On utilise la méthode *MLPClassifier* de *sklearn* pour utiliser un réseau de neurones à deux couches cachées de 200 et 60 neurones :

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(200,60))
mlp.fit(X_train,y_train)
y_mlp = mlp.predict(X_test)

In [ ]:
mlp_score = accuracy_score(y_test, y_mlp)
print(mlp_score)

In [ ]:
pd.crosstab(y_test, y_mlp, rownames=['Reel'], colnames=['Prediction'], margins=True)

La pertinence est assez bonne(0.88) et la matrice de confusion est aussi assez bonne.

## Réseaux denses (Keras/Tensorflow)

Comme l'activation d'un neurone donne une valeur (probabilité) entre 0 ou 1, on code la cible (classes entre 0 et 24) sous la forme d'un vecteur de 0 ou 1 (*one hot encoding*) avec *to_categorical* :

In [ ]:
from keras.utils.np_utils import to_categorical
print(y[0])
y_cat = to_categorical(y)
print(y_cat[0])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=1)

Pour Keras on utilise des tableaux

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

On va utiliser une architecture en *couches* (modèle *Sequential*), avec des couches *denses* :

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

On définit un modèle à deux couches cachées de 200 et 60 neurones  
La dernière couche comporte 25 neurones (le nombre de classes) pour la classification :

In [ ]:
num_classes = y_cat.shape[1]

model = Sequential()
model.add(Dense(200, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

On "compile" le modèle, avec une *categorical_crossentropy* comme mesure de distance (distance probabiliste multi classes)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train = model.fit(X_train , y_train , validation_data=(X_test,y_test), epochs=30, verbose=1)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
def plot_scores(train) :
    accuracy = train.history['accuracy']
    val_accuracy = train.history['val_accuracy']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Score apprentissage')
    plt.plot(epochs, val_accuracy, 'r', label='Score validation')
    plt.title('Scores')
    plt.legend()
    plt.show()

In [ ]:
plot_scores(train)

La pertinence est de 0.87% ce qui est très correct.